In [179]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import lxml

In [48]:
brefids = [
    "colonba01",
    "moronre01",
    "mijarjo01",
    "healeto01",
    "machije01",
    "rodnefe01",
    "castran01",
    "colined01",
    "valdefr01",
    "carigan01",
    "akinke01",
    "mateoju01",
    "gonzaen01",
    "kirkal01",
    "penabr01",
    "sandopa01",
    "whitety01",
    "phegljo01",
    "solando01",
    "callaal01",
    "tejadmi01",
    "naylojo01",
    "vicieda01",
    "ramirha02",
    "astudwi01",
    "fothebo01",
]

In [54]:
targets = []

for player in range(len(brefids)):
    id = brefids[player]
    prepend = "/" + id[0] + "/"
    targets.append(str(prepend) + id)

In [ ]:
root = "https://www.baseball-reference.com/players/"
suffix = ".shtml"
query = root + targets[0] + suffix
response = requests.get(query)
doc = BeautifulSoup(response.text)
page_content = doc.select("#pitching_standard > tfoot > tr:nth-child(2) .right")


In [240]:
metrics = []
vals = []
for row in range(len(page_content)):
    metric = []
    val = []
    stat = []
    metrics.append(page_content[row]['data-stat'])
    vals.append(page_content[row].text)

stats = pd.DataFrame({
    'metrics': metrics,
    'values': vals
})

stats.set_index('metrics').T

metrics,W,L,win_loss_perc,earned_run_avg,G,GS,GF,CG,SHO,SV,...,WP,batters_faced,earned_run_avg_plus,fip,whip,hits_per_nine,home_runs_per_nine,bases_on_balls_per_nine,strikeouts_per_nine,strikeouts_per_base_on_balls
values,15,11,.568,4.12,34,34,0,2,1,0,...,3,892,106,4.15,1.312,9.3,1.1,2.5,6.6,2.67
